This is after we got a 0.78 in the last run on 8th march.
MODEL

This model got 0.84517

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Pre-activation basic block with optional dropout
class PreActBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, out_planes, stride=1, drop_prob=0.0):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.drop_prob = drop_prob

        # If dimensions change, use a projection for the shortcut.
        if stride != 1 or in_planes != out_planes:
            self.shortcut = nn.Conv2d(in_planes, out_planes, kernel_size=1,
                                      stride=stride, bias=False)
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out) if not isinstance(self.shortcut, nn.Identity) else x
        out = self.conv1(out)
        out = F.relu(self.bn2(out))
        if self.drop_prob > 0:
            out = F.dropout(out, p=self.drop_prob, training=self.training)
        out = self.conv2(out)
        out += shortcut
        return out

# RobustResNet64-like model for CIFAR-like images.
# Total layers: 1 conv + 3 groups * (2 convs per block * 10 blocks) + final FC = 1 + 60 + 1 = 62 layers.
class RobustResNet64(nn.Module):
    def __init__(self, num_blocks=10, base_channels=32, num_classes=10, drop_prob=0.1):
        super().__init__()
        self.in_planes = base_channels
        # Initial convolution
        self.conv1 = nn.Conv2d(3, base_channels, kernel_size=3, stride=1,
                               padding=1, bias=False)

        # Three groups (for CIFAR, the convention is 3 groups)
        self.layer1 = self._make_layer(base_channels, num_blocks, stride=1, drop_prob=drop_prob)
        self.layer2 = self._make_layer(base_channels * 2, num_blocks, stride=2, drop_prob=drop_prob)
        self.layer3 = self._make_layer(base_channels * 4, num_blocks, stride=2, drop_prob=drop_prob)

        self.bn = nn.BatchNorm2d(base_channels * 4)
        self.linear = nn.Linear(base_channels * 4, num_classes)

        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, out_planes, blocks, stride, drop_prob):
        layers = []
        # First block in the layer may downsample
        layers.append(PreActBlock(self.in_planes, out_planes, stride=stride, drop_prob=drop_prob))
        self.in_planes = out_planes
        for _ in range(1, blocks):
            layers.append(PreActBlock(self.in_planes, out_planes, stride=1, drop_prob=drop_prob))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn(out))
        # Global average pooling over spatial dimensions
        out = F.adaptive_avg_pool2d(out, 1).view(out.size(0), -1)
        out = self.linear(out)
        return out

if __name__ == "__main__":
    # Create an instance of the model
    model = RobustResNet64(num_blocks=10, base_channels=32, num_classes=10, drop_prob=0.1)
    # Dummy input (e.g. CIFAR: 32x32 images)
    x = torch.randn(2, 3, 32, 32)
    y = model(x)
    print("Output shape:", y.shape)
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total trainable parameters: {total_params:,} (~{total_params/1e6:.2f}M)")


Output shape: torch.Size([2, 10])
Total trainable parameters: 3,799,978 (~3.80M)


In [ ]:
# train.py
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
import numpy as np
import pickle
from torch.utils.data import DataLoader, Dataset
from PIL import Image
#from model import LightResNet18
from tqdm import tqdm

# We'll define a custom collate function for MixUp/CutMix
# randomly picks one method for each batch
def rand_bbox(size, lam):
    W = size[3]
    H = size[2]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def mixup_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def cutmix_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size)
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bby1:bby2, bbx1:bbx2] = x[index, :, bby1:bby2, bbx1:bbx2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size(-1) * x.size(-2)))
    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam

def mixup_cutmix_collate(batch, alpha=1.0, p=0.5):
    # Standard collate
    images, labels = list(zip(*batch))
    images = torch.stack(images, 0)
    labels = torch.tensor(labels, dtype=torch.long)

    # Flip a coin to choose MixUp or CutMix
    if np.random.rand() < p:
        # MixUp
        mixed_x, y_a, y_b, lam = mixup_data(images, labels, alpha)
        return mixed_x, (y_a, y_b, lam, 'mixup')
    else:
        # CutMix
        cutmix_x, y_a, y_b, lam = cutmix_data(images, labels, alpha)
        return cutmix_x, (y_a, y_b, lam, 'cutmix')

def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

class CIFARDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data.reshape(-1, 3, 32, 32).astype("float32") / 255.0
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]*255
        img = img.astype("uint8")
        # shape: (3,32,32) => for PIL => (32,32,3)
        img = Image.fromarray(img.transpose(1,2,0))
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

def load_cifar10_batches(root_dir):
    data_list, labels_list = [], []
    for i in range(1, 6):
        batch_file = f"{root_dir}/data_batch_{i}"
        batch = unpickle(batch_file)
        data_list.append(batch[b'data'])
        labels_list.extend(batch[b'labels'])
    X = np.concatenate(data_list, axis=0)
    y = np.array(labels_list)
    return X, y

def train():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # 1) Load CIFAR-10
    xxxx = """    root_dir = "/content/cifar-10-python/cifar-10-batches-py"
    X, y = load_cifar10_batches(root_dir)
    total_size = len(X)  # 50000
    val_size = 5000
    train_size = total_size - val_size
    indices = np.arange(total_size)
    np.random.shuffle(indices)
    train_idx = indices[:train_size]
    val_idx   = indices[train_size:]

    X_train, y_train = X[train_idx], y[train_idx]
    X_val,   y_val   = X[val_idx],   y[val_idx]"""

    # 2) Transforms: RandAugment or AutoAugment
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandAugment(),  # or AutoAugment()
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616))
    ])
    transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616))
    ])

    #train_dataset = CIFARDataset(X_train, y_train, transform=transform_train)
    #val_dataset   = CIFARDataset(X_val,   y_val,   transform=transform_val)
    train_dataset = datasets.CIFAR10(root='./data', train=True,
                                 download=True, transform=transform_train)
    val_dataset = datasets.CIFAR10(root='./data', train=False,
                                download=True, transform=transform_val)
    # Collate with MixUp & CutMix
    train_loader = DataLoader(train_dataset, batch_size=128,
                              shuffle=True, num_workers=4,
                              collate_fn=lambda b: mixup_cutmix_collate(b, alpha=1.0, p=0.5))
    val_loader   = DataLoader(val_dataset,   batch_size=128,
                              shuffle=False, num_workers=4)

    # 3) Initialize model
    model = RobustResNet64(num_blocks=10, base_channels=32, num_classes=10, drop_prob=0.1).to(device)

    # 4) Loss, Optimizer, Scheduler
    # We'll handle MixUp/CutMix label logic manually, but also do label_smoothing
    base_criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, nesterov=True, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

    best_acc = 0.0
    num_epochs = 300

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_approx = 0.0
        total_approx = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]", leave=False)
        for images, label_info in train_pbar:
            # label_info = (y_a, y_b, lam, method)
            y_a, y_b, lam, method = label_info
            images = images.to(device)
            y_a = y_a.to(device)
            y_b = y_b.to(device)

            optimizer.zero_grad()
            outputs = model(images)

            # MixUp/CutMix combined loss
            loss = lam * base_criterion(outputs, y_a) + (1 - lam) * base_criterion(outputs, y_b)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            # approximate accuracy counting
            correct_approx += (preds == y_a).sum().item() * lam + (preds == y_b).sum().item() * (1 - lam)
            total_approx   += images.size(0)

            train_pbar.set_postfix(loss=f"{loss.item():.3f}")

        scheduler.step()
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc  = 100.0 * correct_approx / total_approx

        # ----- Validation -----
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, pred = torch.max(outputs, 1)
                val_correct += (pred == labels).sum().item()
                val_total   += labels.size(0)
        val_acc = 100.0 * val_correct / val_total

        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {epoch_loss:.4f}, Approx Train Acc: {epoch_acc:.2f}% | "
              f"Val Acc: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print(f"   [*] Saved new best model: Val Acc = {val_acc:.2f}%")

    print(f"Training complete. Best validation accuracy: {best_acc:.2f}%")
    print("Best model saved as best_model.pth")

if __name__ == "__main__":
    train()


Using device: cuda


100%|██████████| 170M/170M [00:01<00:00, 98.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Epoch 1/300 | Train Loss: 2.1139, Approx Train Acc: 23.37% | Val Acc: 45.63%
   [*] Saved new best model: Val Acc = 45.63%


Epoch 2/300 | Train Loss: 1.9044, Approx Train Acc: 37.30% | Val Acc: 57.41%
   [*] Saved new best model: Val Acc = 57.41%


...

Epoch 296/300 | Train Loss: 1.1488, Approx Train Acc: 72.80% | Val Acc: 96.82%


Epoch 297/300 | Train Loss: 1.1769, Approx Train Acc: 70.97% | Val Acc: 96.83%


Epoch 298/300 | Train Loss: 1.1854, Approx Train Acc: 70.24% | Val Acc: 96.82%


Epoch 299/300 | Train Loss: 1.1531, Approx Train Acc: 72.25% | Val Acc: 96.87%


Epoch 300/300 | Train Loss: 1.1613, Approx Train Acc: 71.88% | Val Acc: 96.86%
Training complete. Best validation accuracy: 96.93%
Best model saved as best_model.pth


In [40]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import pickle
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

# Load your RobustResNet64 model here or define it
# from model import RobustResNet64

def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

class CIFARTestDataset(Dataset):
    def __init__(self, data, ids, transform_augmentations=None):
        self.data = data
        self.ids = ids
        self.transform_augmentations = transform_augmentations

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx].astype("uint8")
        img = Image.fromarray(img)  # Convert to PIL Image
        img_id = self.ids[idx]

        all_crops = []
        for transform in self.transform_augmentations:
            augmented_img = transform(img)
            crops = transforms.TenCrop(32)(augmented_img)
            all_crops.extend(crops)

        # Convert crops to tensors and normalize
        tensor_crops = []
        for crop in all_crops:
            tensor = transforms.ToTensor()(crop)
            tensor = transforms.Normalize((0.4914, 0.4822, 0.4465),
                                         (0.2470, 0.2435, 0.2616))(tensor)
            tensor_crops.append(tensor)

        tensor_crops = torch.stack(tensor_crops)
        return tensor_crops, img_id

def inference():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Load model
    model = RobustResNet64(num_blocks=10, base_channels=32, num_classes=10, drop_prob=0.1).to(device)
    model.load_state_dict(torch.load("best_model.pth", map_location=device))
    model.eval()

    # Load test data
    test_file = "/content/cifar_test_nolabel.pkl"
    test_dict = unpickle(test_file)
    test_images = test_dict[b'data'].reshape(-1, 32, 32, 3)
    test_ids = [str(i) for i in range(len(test_images))]

    # Define multiple augmentation pipelines
    augmentation_transforms = [
        transforms.Compose([
            transforms.Pad(4),
        ]),
        transforms.Compose([
            transforms.Pad(4),
            transforms.RandomHorizontalFlip(p=1.0),
        ]),
        transforms.Compose([
            transforms.Pad(4),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        ]),
        transforms.Compose([
            transforms.Pad(4),
            transforms.RandomRotation(10),
        ]),
        transforms.Compose([
            transforms.Pad(4),
            transforms.RandomRotation(350),
        ]),
    ]

    test_dataset = CIFARTestDataset(test_images, test_ids,
                                   transform_augmentations=augmentation_transforms)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

    predictions = []
    image_ids = []
    with torch.no_grad():
        for imgs, ids in test_loader:
            # imgs shape: (B, num_augmentations*10, C, H, W)
            bs, num_crops, C, H, W = imgs.size()
            imgs = imgs.view(-1, C, H, W).to(device)

            outputs = model(imgs)
            outputs = outputs.view(bs, num_crops, -1)
            avg_outputs = outputs.mean(dim=1)
            probs = F.softmax(avg_outputs, dim=1)
            _, predicted = torch.max(probs, 1)

            predictions.extend(predicted.cpu().numpy().tolist())
            image_ids.extend(ids)

    submission_df = pd.DataFrame({"ID": image_ids, "Labels": predictions})
    submission_df.to_csv("submission.csv", index=False)
    print("Submission file saved as submission.csv")

if __name__ == "__main__":
    inference()

Using device: cuda


<ipython-input-40-e42665737abd>:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth", map_location=device))
/usr/local/lib/pyt

Submission file saved as submission.csv


In [41]:
import pandas as pd

# Load and inspect submission file
submission_df = pd.read_csv("submission.csv")
print(submission_df.head())  # Show first few rows
print(submission_df["Labels"].value_counts())  # Show label distribution


   ID  Labels
0   0       6
1   1       1
2   2       8
3   3       6
4   4       9
Labels
3    1077
5    1040
1    1035
8    1025
9    1016
7    1016
2     993
6     958
4     939
0     901
Name: count, dtype: int64
